In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
#from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.model_selection import train_test_split


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/cat-in-the-dat-ii/sample_submission.csv
/kaggle/input/cat-in-the-dat-ii/test.csv
/kaggle/input/cat-in-the-dat-ii/train.csv


In [2]:
def auc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return metrics.roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)


In [3]:
train = pd.read_csv("../input/cat-in-the-dat-ii/train.csv")#.sample(n=30000, random_state=1)
test = pd.read_csv("../input/cat-in-the-dat-ii/test.csv")#.sample(n=1000, random_state=1)
sample = pd.read_csv("../input/cat-in-the-dat-ii/sample_submission.csv")

df_train = train 
df_test = test
print(train.shape, test.shape)


(30000, 25) (1000, 24)


In [ ]:
#print(df_test.head())
df_test = pd.DataFrame(df_test)
df_test = df_test.fillna(df_test.mean())
df_test = df_test.fillna('Empty999')
#print(df_test.head())


In [4]:
#transforming all features together 
df_test["target"] = -1
data = pd.concat([df_train, df_test]).reset_index(drop=True)
print(data.shape ,df_train.shape, df_test.shape)
data.dropna(inplace = True)
print(data.shape ,df_train.shape, df_test.shape)
features = [x for x in df_train.columns if x not in ["id", "target"]]

for feat in features:
    lbl_enc = preprocessing.LabelEncoder()
    data[feat] = lbl_enc.fit_transform(data[feat].fillna("-1").astype(str).values)
    

(31000, 25) (30000, 25) (1000, 25)
(15265, 25) (30000, 25) (1000, 25)


In [5]:
#Exploration: eliminate categorial variables with many classes (for now, might add them back later)
#print(data.head())
#for feat in features:
    #print(feat, data[feat].value_counts().shape)

#eliminate nom_5, nom_6, nom_9    
did = data['id'].values
dtarget = data['target'].values
data.drop(['nom_5','nom_6','nom_9','id','target'],axis=1,inplace=True)


#Estimate number of columns
for feat in data.columns:
    count = data[feat].value_counts().shape
    #print(feat, count)

In [6]:
mlb = preprocessing.OneHotEncoder(handle_unknown='ignore')
data_new = mlb.fit_transform(data.astype(str))
print(data_new.shape)
print(data.shape)

(15265, 742)
(15265, 20)


In [7]:
data_new = pd.DataFrame(data_new.toarray())
data_new['id'] = did
data_new['target'] = dtarget
print(data_new.shape)

(15265, 744)


In [8]:
train = data_new[data_new.target != -1].reset_index(drop=True)
test = data_new[data_new.target == -1].reset_index(drop=True)

print(train.shape,test.shape, data_new.shape)


(14785, 744) (480, 744) (15265, 744)


In [11]:
nb = 30000 #(30k)
train1 = data_new[data_new.target == 1].reset_index(drop=True)
train0 = data_new[data_new.target == 0].reset_index(drop=True)
train1 = train1.sample(n=nb, random_state=5)
train0 = train0.sample(n=nb, random_state=6)
train = pd.concat([train1, train0]).reset_index(drop=True)
train = train.sample(frac=1).reset_index(drop=True)

test = data_new[data_new.target == -1].reset_index(drop=True)
#test = test.sample(n=int(test.shape[0]*0.1), random_state=1)
#test_data = [test.loc[:, features].values[:, k] for k in range(test.loc[:, features].values.shape[1])]

print('First time consuming fitting',train.shape, test.shape)
y = np.array(train.target)
X = train.drop(['target','id'], axis  = 1)
X_ = test.drop(['target','id'], axis  = 1)
X = X.to_numpy()
X_ = X_.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

First time consuming fitting (5000, 744) (480, 744)


In [ ]:
#from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import StratifiedShuffleSplit
#from sklearn.svm import SVC


#param_grid = [
#  {'C': [0.1, 0.2, 0.4, 0.5, 1, 1.5,10], 'kernel': ['linear']},
#  {'C': [0.1, 0.2, 0.4, 0.5, 1, 1.5,10], 'gamma':[0.0001, 0.001, 0.01,0.05,0.08,0.1, 0.15, 0.5, 1], 'kernel': ['rbf']},
#]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

#grid = GridSearchCV(SVC(class_weight={1: 5}), param_grid=param_grid, cv=cv)
#grid.fit(X_train, y_train)

#print("The best parameters are %s with a score of %0.2f"
#      % (grid.best_params_, grid.best_score_))
#grid.cv_results_.keys()#['mean_test_score']     
#grid.cv_results_['mean_test_score']   

In [ ]:
#Model 1: 0.67860 (V8) (only 100000 samples)
model1 = False
if model1: 
    clf = svm.SVC(kernel='rbf',gamma=0.1,C=0.3)
    clf.fit(X_train, y_train)
    y_val = clf.predict(X_test)
    y_train_val = clf.predict(X_train)
    test_preds =  clf.predict(X_)

In [ ]:
#Model 2: 0.66 (V11)
model2 = False
if model2: 
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier(n_estimators=500, max_depth=30, class_weight={1: 5}, random_state = 42)
    rf.fit(X_train, y_train)
    y_val = rf.predict(X_test)
    y_train_val = rf.predict(X_train)
    test_preds =  rf.predict(X_)


In [ ]:
#Model 3: 0.62027 (v15)
model3 = False
if model3 and model2:
    fi = pd.DataFrame({'col':np.arange(0,len(rf.feature_importances_)), 'fi': rf.feature_importances_})
    fi.sort_values(by=['fi'], inplace=True,ascending=False)
    fi = fi.iloc[0:20,:]

    train = data_new[data_new.target != -1].reset_index(drop=True)
    train = train.sample(n=int(train.shape[0]*0.8), random_state=10)
    test = data_new[data_new.target == -1].reset_index(drop=True)
    #test_data = [test.loc[:, features].values[:, k] for k in range(test.loc[:, features].values.shape[1])]
    print(train.shape)
    y = np.array(train.target)
    X = train.drop(['target','id'], axis  = 1)
    X_ = test.drop(['target','id'], axis  = 1)
    X = X[fi.col]
    X_ = X_[fi.col]

    X = X.to_numpy()
    X_ = X_.to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    print(X_train.shape)
    clf = svm.SVC(kernel='rbf',gamma=0.1,C=0.3, class_weight={1: 5})
    clf.fit(X_train, y_train)
    y_val = clf.predict(X_test)
    y_train_val = clf.predict(X_train)
    test_preds =  clf.predict(X_)

In [12]:
#model 4: NN
model4 = True

if model4: 
    import tensorflow as tf
    from tensorflow import keras

    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(X_train.shape[1],)),
        keras.layers.Dense(16, activation=tf.nn.relu),
        keras.layers.Dense(16, activation=tf.nn.relu),
        keras.layers.Dense(1, activation=tf.nn.sigmoid),
    ])
    #model = keras.Sequential([
    #keras.layers.Flatten(input_shape=(28, 28)),
    #keras.layers.Dense(128, activation='relu'),
    #keras.layers.Dense(10)
    #])
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=50, batch_size=1)
    #test_loss, test_acc = model.evaluate(X_test, y_test)
    y_val = model.predict(X_test)
    y_train_val = model.predict(X_train)
    test_preds =  model.predict(X_)


Train on 3350 samples
Epoch 1/50
3350/3350 [==============================] - 6s 2ms/sample - loss: 0.6346 - accuracy: 0.6496
Epoch 2/50
3350/3350 [==============================] - 6s 2ms/sample - loss: 0.5480 - accuracy: 0.7266
Epoch 3/50
3350/3350 [==============================] - 5s 2ms/sample - loss: 0.5013 - accuracy: 0.7537
Epoch 4/50
3350/3350 [==============================] - 5s 2ms/sample - loss: 0.4403 - accuracy: 0.7925
Epoch 5/50
3350/3350 [==============================] - 5s 2ms/sample - loss: 0.3722 - accuracy: 0.8349
Epoch 6/50
3350/3350 [==============================] - 5s 2ms/sample - loss: 0.3023 - accuracy: 0.8722
Epoch 7/50
3350/3350 [==============================] - 5s 2ms/sample - loss: 0.2248 - accuracy: 0.9116
Epoch 8/50
3350/3350 [==============================] - 5s 2ms/sample - loss: 0.1637 - accuracy: 0.9409
Epoch 9/50
3350/3350 [==============================] - 5s 2ms/sample - loss: 0.1100 - accuracy: 0.9654
Epoch 10/50
3350/3350 [===================

NameError: name 'clf' is not defined

In [ ]:
#clf = svm.SVC(kernel='rbf',gamma=0.1,C=0.3, class_weight={1: 5})
#clf.fit(X_train, y_train)
#y_val = clf.predict(X_test)
#y_train_val = clf.predict(X_train)
#test_preds =  clf.predict(X_)

In [20]:
print(confusion_matrix(y_train,y_train_val.astype(int)))
print(confusion_matrix(y_test,y_val.astype(int)))
print("Overall AUC={}".format(metrics.roc_auc_score(y_test, y_val.astype(int))))

[[1664    0]
 [   7 1679]]
[[684 152]
 [515 299]]
Overall AUC=0.5927518427518427


In [ ]:
#test_preds /= 50
test_ids = test.id.values
print("Saving submission file")
submission = pd.DataFrame.from_dict({
    'id': test_ids,
    'target': test_preds
})
submission.to_csv("submission.csv", index=False)